A co-visitation matrix is essentially an "analog" approximation to matrix factorization! I talk a bit more about this idea here: [💡 What is the co-visitation matrix, really?](https://www.kaggle.com/competitions/otto-recommender-system/discussion/365358).

But matrix factorization has a lot of advantages as compared to co-visitation matrices. First of all, it can make better use of data -- it operates on the notion of similarity between categories. We can construct a more powerful representation if our model understands that aid `1` is similar to aid `142` as opposed to it treating each aid as an atomic entity (this is the jump from unigram/bigram/trigram models to word2vec in NLP).

Let us thus train a matrix factorization model and replace the co-visitation matrices with it!

Now, I don't expect that the first version of the model will be particularly well tuned. There has already been a lot of work put into co-visitation matrices and in the later versions we work off 3 different matrices, one for each category of actions! A similar progression can and will happen with matrix factorization 🙂 This notebook hopefully will enable us to jumpstart this type of exploration 🙂

To streamline the work, we will use data in `parquet` format. (Here is the notebook [💡 [Howto] Full dataset as parquet/csv files](https://www.kaggle.com/code/radek1/howto-full-dataset-as-parquet-csv-files) and here is [the most up-to-date version of the dataset](https://www.kaggle.com/datasets/radek1/otto-full-optimized-memory-footprint), no need for dealing with `jasonl` files and the associated mess any longer! Please upvote if you find this useful!)

For data processing we will use [polars](https://www.pola.rs/). `Polars` has a much smaller memory footprint than `pandas` and is quite fast. Plus it has really clean, intuitive API.

Let's get to work! 🙂

**UPDATE:** [@CPMP](https://www.kaggle.com/cpmpml) ported this notebook to run fully on the GPU! 🥳 This is awesome as it can allow you to experiment and ensemble models much faster 🔥 The other notebook also demonstrates how to accelerate a workflow very important to RecSys (and at the heart of the predictions in the notebook you are reading now) -- the nearest neighbor search algorithm. By running it on the GPU not only do you get significantly better results (you don't have to rely on approximate nearest neighbor search anymore, there is enough compute to develiver __true nearest neighbor search faster than you could run ANN on the CPU!__)

Please find the GPU accelerated notebook here: [💡Matrix Factorization with GPU](https://www.kaggle.com/code/cpmpml/matrix-factorization-with-gpu)

And if you would be interested in a further discussion of NN (nearest neighbor) vs ANN (approximate nearest neighbor) and running them the CPU/GPU, please see my post [here](https://www.kaggle.com/competitions/otto-recommender-system/discussion/371111) (though it is a bit outdated as `cuml` has a nicer API).


# Data Preprocessing

In [2]:
import polars as pl

train = pl.read_parquet('/Users/chronus/Data/code/Git-repo/machine learning/data/train.parquet')
test = pl.read_parquet('/Users/chronus/Data/code/Git-repo/machine learning/data/test.parquet')

We need to create `aid-aid` pairs to train our matrix factorization model!

Let's us grab the pairs both from the train and test set.

In [3]:
%%time

train_pairs = (pl.concat([train, test])
    .groupby('session').agg([
        pl.col('aid'),
        pl.col('aid').shift(-1).alias('aid_next')
    ])
    .explode(['aid', 'aid_next'])
    .drop_nulls()
)[['aid', 'aid_next']]

<timed exec>:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.


CPU times: user 15.2 s, sys: 4.72 s, total: 19.9 s
Wall time: 13.4 s


In [4]:
train_pairs.shape[0] / 1_000_000

209.072637

That is 209 million pairs created in 40 seconds without running out of RAM! 🙂 Not too bad

In [5]:
train_pairs.head()

aid,aid_next
i32,i32
1590807,1228619
1228619,1590807
1590807,1590807
1590807,1590807
1590807,1590807


Let's see what is the cardinality of our aids -- we will need this to create the embedding layer.

In [6]:
cardinality_aids = max(train_pairs['aid'].max(), train_pairs['aid_next'].max())
cardinality_aids

1855602

We will have up to `1855602` -- that is a lot! But our matrix factorization model will be able to handle this.

Let's construct a `PyTorch` dataset and `dataloader`.

In [7]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ClicksDataset(Dataset):
    def __init__(self, pairs):
        self.aid1 = pairs['aid'].to_numpy()
        self.aid2 = pairs['aid_next'].to_numpy()
    def __getitem__(self, idx):
        aid1 = self.aid1[idx]
        aid2 = self.aid2[idx]
        return [aid1, aid2]
    def __len__(self):
        return len(self.aid1)

train_ds = ClicksDataset(train_pairs[:-10_000_000])
valid_ds = ClicksDataset(train_pairs[10_000_000:])

Let us see how quickly we can iterate over a single epoch with a batch size of `65536`.

In [8]:
train_ds = ClicksDataset(train_pairs)
train_dl_pytorch = DataLoader(train_ds, 65536, True, num_workers=2)

In [8]:
%%time

for batch in train_dl_pytorch:
    aid1, aid2 = batch[0], batch[1]

CPU times: user 2min 44s, sys: 25 s, total: 3min 9s
Wall time: 11min 48s


Oh dear, that took forever! Mind you, were are not doing anything here, apart from iterating over the dataset for a single epoch (and that is without validation!).

The reason this is taking so long is that indexing into the the arrays and collating results into batches is very computationally expensive.

There are ways to work around this but they require writing a lot of code (you could use the iterable-style dataset). And still our solution wouldn't be particularly well optimized.

Let us do something else instead!

We will use a brand new [Merlin Dataloader](https://github.com/NVIDIA-Merlin/dataloader). It is a library that my team launched just a couple of days ago 🙂

Now this library shines when you have a GPU, which is what you generally want when training DL models. But, alas, Kaggle gives you only 13 GB of RAM on a kernel with a GPU, and that wouldn't allow us to process our dataset!

Let's see how far we can get with CPU only.

In [9]:
!pip install merlin-dataloader==0.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.7/104.7 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Installing build dependencies ... - \ | d

In [10]:
from merlin.loader.torch import Loader 

We can read data directly from the disk -- even better!

Let's write our datasets to disk.

In [9]:
train_pairs[:-10_000_000].to_pandas().to_parquet('train_pairs.parquet')
train_pairs[-10_000_000:].to_pandas().to_parquet('valid_pairs.parquet')

In [10]:
from merlin.loader.torch import Loader 
from merlin.io import Dataset

train_ds = Dataset('train_pairs.parquet')
train_dl_merlin = Loader(train_ds, 65536, True)

/Users/chronus/miniforge3/envs/ml/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/Users/chronus/miniforge3/envs/ml/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")
/Users/chronus/miniforge3/envs/ml/lib/python3.10/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [11]:
%%time

for batch, _ in train_dl_merlin:
    aid1, aid2 = batch['aid'], batch['aid_next']

CPU times: user 11.7 s, sys: 4.78 s, total: 16.5 s
Wall time: 18.6 s


That is much better 🙂. Let's train our matrix factorization model!

In [12]:
class MatrixFactorization(nn.Module):
    def __init__(self, n_aids, n_factors):
        super().__init__()
        self.aid_factors = nn.Embedding(n_aids, n_factors, sparse=True)
        
    def forward(self, aid1, aid2):
        aid1 = self.aid_factors(aid1)
        aid2 = self.aid_factors(aid2)
        
        return (aid1 * aid2).sum(dim=1)
    
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

valid_ds = Dataset('valid_pairs.parquet')
valid_dl_merlin = Loader(valid_ds, 65536, True)

/Users/chronus/miniforge3/envs/ml/lib/python3.10/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


In [13]:
from torch.optim import SparseAdam

num_epochs=1
lr=0.1

model = MatrixFactorization(cardinality_aids+1, 32)
optimizer = SparseAdam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

In [14]:
%%time

for epoch in range(num_epochs):
    for batch, _ in train_dl_merlin:
        model.train()
        losses = AverageMeter('Loss', ':.4e')
            
        aid1, aid2 = batch['aid'], batch['aid_next']
        output_pos = model(aid1, aid2)
        output_neg = model(aid1, aid2[torch.randperm(aid2.shape[0])])
        
        output = torch.cat([output_pos, output_neg])
        targets = torch.cat([torch.ones_like(output_pos), torch.zeros_like(output_pos)])
        loss = criterion(output, targets)
        losses.update(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    model.eval()
    
    with torch.no_grad():
        accuracy = AverageMeter('accuracy')
        for batch, _ in valid_dl_merlin:
            aid1, aid2 = batch['aid'], batch['aid_next']
            output_pos = model(aid1, aid2)
            output_neg = model(aid1, aid2[torch.randperm(aid2.shape[0])])
            accuracy_batch = torch.cat([output_pos.sigmoid() > 0.5, output_neg.sigmoid() < 0.5]).float().mean()
            accuracy.update(accuracy_batch, aid1.shape[0])
            
    print(f'{epoch+1:02d}: * TrainLoss {losses.avg:.3f}  * Accuracy {accuracy.avg:.3f}')

01: * TrainLoss 1.178  * Accuracy 0.687
CPU times: user 8min 29s, sys: 10min 32s, total: 19min 1s
Wall time: 4min 19s


Let's grab the embeddings!

In [15]:
embeddings = model.aid_factors.weight.detach().numpy()

And construct create the index for approximate nearest neighbor search.

In [17]:
%%time

from annoy import AnnoyIndex

index = AnnoyIndex(32, 'euclidean')
for i, v in enumerate(embeddings):
    index.add_item(i, v)
    
index.build(10)

CPU times: user 36.3 s, sys: 1.03 s, total: 37.4 s
Wall time: 14.1 s


True

Now for any `aid`, we can find its nearest neighbor!

In [18]:
index.get_nns_by_item(123, 10)

[123, 539087, 1007778, 405376, 605734, 704554, 57551, 1115280, 974562, 324669]

Let's create a submission! 🙂

In [20]:
import pandas as pd
import numpy as np

from collections import defaultdict

sample_sub = pd.read_csv('/Users/chronus/Data/code/Git-repo/machine learning/sample_submission.csv')

session_types = ['clicks', 'carts', 'orders']
test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)

labels = []

type_weight_multipliers = {0: 1, 1: 6, 2: 3}
for AIDs, types in zip(test_session_AIDs, test_session_types):
    if len(AIDs) >= 20:
        # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(AIDs,weights,types): 
            aids_temp[aid]+= w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        labels.append(sorted_aids[:20])
    else:
        # here we don't have 20 aids to output -- we will use approximate nearest neighbor search and our embeddings
        # to generate candidates!
        AIDs = list(dict.fromkeys(AIDs[::-1]))
        
        # let's grab the most recent aid
        most_recent_aid = AIDs[0]
        
        # and look for some neighbors!
        nns = index.get_nns_by_item(most_recent_aid, 21)[1:]
                        
        labels.append((AIDs+nns)[:20])

Let's now pull it all together and write to a file,

In [21]:
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

prediction_dfs = []

for st in session_types:
    modified_predictions = predictions.copy()
    modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
    prediction_dfs.append(modified_predictions)

submission = pd.concat(prediction_dfs).reset_index(drop=True)
submission.to_csv('submission.csv', index=False)

And we are done!


**If you like this notebook, please smash the upvote button! Thank you! 😊**

There are many ways in which this can be expanded:
* we can train on the GPU
* we can train for longer
* maybe we would get better results if we were to filter our train data by type?
* should we train only on adjacent aids? maybe we should expand the neighborhood we train on

We can keep asking ourselves many questions like this 🙂 Now we have a framework to start answering them!

Thank you for reading! Happy Kaggling! 🙌